In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from Crypto.Cipher import AES
import IPython.display
import glob
import os



In [ ]:

home_dir = os.path.expanduser("~")
data_dir = os.path.join(home_dir, ".tokens")
dl_dir = "F:\\downloads\\"
# get the latest CSV file
os.chdir(dl_dir)
csvfiles = glob.glob(os.path.join(".", '*.csv'))
sorted_by_mtime_ascending = sorted(csvfiles, key=lambda t: os.stat(t).st_mtime)
historic = [ x for x in sorted_by_mtime_ascending if "glucose" in x ]
print(historic)
csvfile = historic[-1]
print(csvfile)
historicCSV = pd.read_csv(csvfile, on_bad_lines='skip',  header=1) 
# need to specify header=1 to make second row specify the columns
historicCSV.head()

In [ ]:
#ordersCSV['Price'] = ordersCSV['Price'].str.extract(r'^(\d+(\.\d+)?)$')
historicCSV = historicCSV.rename(columns={
    'Device Timestamp': 'timestamp',
    'Historic Glucose mg/dL': 'glucoseHist',
    'Scan Glucose mg/dL': 'glucoseScan'
})
historicCSV.head()

In [ ]:
historicCSV['glucoseHist'] = historicCSV['glucoseHist'].astype(str).replace('[^0-9\.]', '', regex=True)
historicCSV['glucoseHist'] = pd.to_numeric(historicCSV['glucoseHist'])
historicCSV['glucoseScan'] = historicCSV['glucoseScan'].astype(str).replace('[^0-9\.]', '', regex=True)
historicCSV['glucoseScan'] = pd.to_numeric(historicCSV['glucoseScan'])
historicCSV.head()

In [ ]:
gluDF = historicCSV.copy()
gluDF = gluDF[['timestamp', 'glucoseScan', 'glucoseHist', 'Notes']]
gluDF.loc[:, 'timestamp'] = pd.to_datetime(gluDF['timestamp'])
gluDF

In [ ]:
# notes have a timestamp but doesn't have glucose reading so make on up
gluDF.sort_values(by='timestamp', inplace=True)
gluDF.reset_index(drop=True, inplace=True)
for index, row in gluDF.iterrows():
    print(f': {row["timestamp"]}, Scan: {row["glucoseScan"]}, Hist: {row["glucoseHist"]}')

In [ ]:
# Function to check if a value is numeric
def is_numeric(value):
    return isinstance(value, (int, float, np.integer, np.floating)) and not np.isnan(value)

gluDF['glucose'] = gluDF.apply(lambda row: row['glucoseScan'] if is_numeric(row['glucoseScan']) else row['glucoseHist'], axis=1)
for index, row in gluDF.iterrows():
    print(f': {row["timestamp"]}, {row["glucose"]}, Hover Comment: {row["Notes"]}')

In [ ]:
gluDF.dropna(subset=['glucose','Notes'], how='all', inplace=True)
for index, row in gluDF.iterrows():
    print(f': {row["timestamp"]}, {row["glucose"]}, Hover Comment: {row["Notes"]}')

In [ ]:
gluDF['glucose'].fillna(gluDF['glucose'].rolling(window=2, min_periods=1, center=True).mean(), inplace=True)
gluDF['Notes'].fillna('', inplace=True)
gluDF

In [ ]:
import plotly.express as px

fig = px.line(gluDF, x='timestamp', y='glucose', title='Zoomable Time Series Plot', hover_name='Notes')

# Enable zooming and panning
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show()

In [15]:
import plotly.express as px

# Sample DataFrame with timestamps, values, and hover comments
data = {
    'timestamp': pd.date_range(start='2023-09-25', periods=10, freq='D'),
    'value': [10, 20, 15, 30, 25, 40, 35, 50, 45, 60],
    'hover_comment': ['Good', 'Low', '', 'High', 'Medium', 'Good', 'Medium', 'High', 'Low', 'Good']
}

df = pd.DataFrame(data)

# Create a Plotly Scatter plot
fig = px.scatter(gluDF, x='timestamp', y='glucose', hover_data=['Notes'], title='Highlighted Portions')

highlighted_color = 'red'
default_color = 'blue'
# Define marker settings for specific hover comments
mask = gluDF['Notes'].str.len() > 0
mask

2        True
3        True
4       False
5       False
6       False
        ...  
1693    False
1702    False
1705     True
1706     True
1707     True
Name: Notes, Length: 1460, dtype: bool

for index, row in gluDF.iterrows():
    print(f': {row["timestamp"]}, {row["glucose"]}, Hover Comment: {row["Notes"]}')

# Debugging: Print hover text and timestamp for comments with at least 4 characters
debug_df = gluDF[mask]
for index, row in debug_df.iterrows():
    print(f'Timestamp: {row["timestamp"]}, {row["glucose"]}, Hover Comment: {row["Notes"]}')

In [16]:

fig.add_scatter(
    x=gluDF[mask]['timestamp'],
    y=gluDF[mask]['glucose'],
    mode='markers',
    name=f'Long Comments',
    marker=dict(size=10, color=highlighted_color),
    #hovertemplate='%{text}<extra></extra>',  # Hide hover label
    text=gluDF[mask]['Notes']
)

'''
# Set marker settings for other points
fig.add_scatter(
    x=gluDF[~mask]['timestamp'],
    y=gluDF[~mask]['glucose'],
    mode='markers',
    name='Other',
    marker=dict(size=10, color=default_color),
    #hovertemplate='%{text}<extra></extra>',  # Hide hover label
    text=''
)
'''

# Show the plot
fig.show()

In [ ]:
highlighted_color = 'red'
default_color = 'blue'

for comment in highlighted_comments:
    mask = gluDF['Notes'] == comment
    fig.add_scatter(
        x=gluDF[mask]['timestamp'],
        y=gluDF[mask]['glucose'],
        mode='markers',
        name=f'{comment} Highlight',
        marker=dict(size=10, color=highlighted_color),
        #hovertemplate='%{text}<extra></extra>',  # Hide hover label
        text=gluDF[mask]['Notes']
    )

# Set marker settings for other points
fig.add_scatter(
    x=gluDF[~gluDF['Notes'].isin(highlighted_comments)]['timestamp'],
    y=gluDF[~gluDF['Notes'].isin(highlighted_comments)]['glucose'],
    mode='markers',
    name='Other',
    marker=dict(size=10, color=default_color),
    #hovertemplate='%{text}<extra></extra>',  # Hide hover label
    text=gluDF[~gluDF['Notes'].isin(highlighted_comments)]['Notes']
)

# Show the plot
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a line plot
plt.figure(figsize=(10, 6))  # Set the figure size
plt.plot(gluDF['timestamp'], gluDF['glucose'], marker='o', linestyle='-')

# Set labels and title
plt.xlabel('Date and Time')
plt.ylabel('Value')
plt.title('Time Series Data')

# Rotate x-axis labels for readability (optional)
plt.xticks(rotation=45)

# Show the plot
plt.grid(True)  # Add grid lines (optional)
plt.tight_layout()  # Ensure labels fit within the figure

plt.show()